In [10]:
import torch

x = torch.arange(4.0)
x.requires_grad_(True)
x.grad


In [11]:

y = 2*torch.dot(x,x)
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [8]:
a = torch.tensor(1.0)
b = torch.tensor(2.0)
a.requires_grad_(True),b.requires_grad_(True)
z = a**2+4*b
z


z.backward()
a.grad

b.grad

tensor(4.)

## 梯度清零 x.grad.zero_()
在 PyTorch 中，`.grad.zero_()` 方法用于将张量的梯度归零。这通常用在神经网络的训练循环中，在每次进行梯度计算之前清零累积的梯度。具体到 `x.grad.zero_()`，这个方法将 `x` 的梯度重置为零。

神经网络训练中为什么需要归零梯度：

1. **累积梯度**：PyTorch 默认会累积梯度，这意味着每次调用 `.backward()` 时，梯度不是被覆盖，而是累加到已存在的梯度上。这在某些情况（如循环神经网络）是有用的，但在大多数情况下我们希望每次迭代只处理当前批次的梯度。

2. **清除历史梯度**：如果不清零，梯度会持续累积，导致每一次迭代的梯度都是基于之前所有迭代的梯度之和，这会妨碍正确的梯度计算和网络的有效训练。

3. **每次迭代独立**：通过在每个训练迭代开始前归零梯度，确保每个迭代步骤的梯度计算是独立的，只反映当前批次数据的损失梯度。

例子：
```python
for epoch in range(num_epochs):
    for data, target in dataset:
        optimizer.zero_grad()  # 用优化器清零所有模型参数的梯度
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新参数
```

在这个例子中，`optimizer.zero_grad()` 调用确保了每次进入一个新的数据批次时，之前批次的梯度不会影响当前批次的梯度计算。在一些情况下，你可能会看到类似 `x.grad.zero_()` 的代码，这是直接对特定张量的梯度进行归零，而 `optimizer.zero_grad()` 是更常用的做法，它会清零优化器中所有参数的梯度。

PyTorch 之所以设计为默认累积梯度，主要是基于以下几个原因：

1. **灵活性**：累积梯度提供了更大的灵活性。在某些情况下，如训练大型网络时内存不足，或者在特定的模型（如循环神经网络）中，可能需要跨多个批次累积梯度。这样可以在不增加内存压力的情况下有效地处理大型数据集或复杂模型。

2. **支持更复杂的优化策略**：在某些高级优化策略中，可能需要在更新模型参数之前考虑多个前向传播的结果。累积梯度的特性允许开发者实现这些更复杂的策略。

3. **效率和性能**：在某些情况下，比如分布式训练，累积梯度可以减少通信次数，从而提高效率。在每个节点上累积更多的梯度数据，然后进行一次更大的参数更新，可以减少网络通信带来的开销。

4. **模型架构的灵活性**：某些模型架构，特别是那些有多个输出或多个任务（如多任务学习）的模型，可能需要根据不同的损失函数累积梯度。这种设计使得在单次传递中对多个任务的梯度进行累积成为可能。

总的来说，PyTorch 选择累积梯度是为了提供更大的灵活性和适应更广泛的使用场景，尤其是在复杂的训练情况和高级优化策略中。但这也意味着在标准的训练循环中，开发者需要显式地在每个迭代步骤开始前清除梯度。

In [20]:
# x.grad.zero_()
y = x.sum()
y.backward()
x.grad


tensor([1., 1., 1., 1.])

In [42]:
# 如果x为向量
x = torch.arange(4.0)
x.requires_grad_(True)

y = x*x

y.sum().backward()
#y.backward(torch.ones(len(x)))
x.grad 


tensor([0., 2., 4., 6.])

# 对于非标量y，使用sum求其梯度
假设 $\mathbf{x}$ 是一个向量，其元素为 $x_1, x_2, ..., x_n$。接着定义 $\mathbf{y}$ 作为 $\mathbf{x}$ 的元素平方的向量，即 $y_i = x_i^2$ 对于所有 $i$。然后计算 $\mathbf{y}$ 的元素和：
$$z = \sum_{i=1}^{n} y_i$$
我们关心的是 $z$ 相对于 $\mathbf{x}$ 的梯度。首先，考虑 $z$ 相对于 $\mathbf{y}$ 的梯度。由于 $z$ 是 $\mathbf{y}$ 的元素和，所以 $z$ 相对于每个 $y_i$ 的偏导数是 1：
$$\frac{\partial z}{\partial y_i} = 1$$
接下来，根据链式法则，$z$ 相对于 $x_i$ 的梯度可以表示为：
$$\frac{\partial z}{\partial x_i} = \frac{\partial z}{\partial y_i} \cdot \frac{\partial y_i}{\partial x_i}$$
由于 $y_i = x_i^2$，我们有：
$$\frac{\partial y_i}{\partial x_i} = 2x_i$$
因此：
$$\frac{\partial z}{\partial x_i} = 1 \cdot 2x_i = 2x_i$$
当执行 `y.sum().backward()` 操作时，PyTorch 会自动进行这些计算，结果就是 $\mathbf{x}$ 的梯度（即 $\mathbf{x}$.grad）将包含 $2 \mathbf{x}$ 的值。这意味着 $\mathbf{x}$.grad 中的每个元素都是对应的 $x_i$ 的两倍。

In [25]:
torch.ones(len(x))

tensor([1., 1., 1., 1.])

In [43]:
x = torch.arange(4.0)
x.requires_grad_(True)
y = x*x+4
u = y.detach()
z = u*x

z.sum().backward()
x.grad


tensor([ 3.,  4.,  7., 12.])